# PEPS

In [1]:
import quimb.tensor as qtn
import symmray as sr
import numpy as np
from vmc_torch.fermion_utils import from_quimb_config_to_netket_config, from_netket_config_to_quimb_config, from_spinful_ind_to_charge, fPEPS
seed = np.random.randint(0, 2**32)  # Random seed for reproducibility
D = 8
Lx, Ly = 4, 4
nelec = int(Lx*Ly)
nu, nd = int(nelec/2), int(nelec/2)  # number of up and down fermions for charge configuration
edges = qtn.edges_2d_square(Lx, Ly, cyclic=False)
site_info = sr.parse_edges_to_site_info(
    edges,
    D,
    phys_dim=4,
    site_ind_id="k{},{}",
    site_tag_id="I{},{}",
)

peps = qtn.TensorNetwork()
rng = np.random.default_rng(seed)
charge_config = np.zeros(Lx*Ly, dtype=int)

charge_config_netket = from_quimb_config_to_netket_config(charge_config)
charge_config_netket_u = charge_config_netket[:len(charge_config_netket)//2]  # up spins
charge_config_netket_d = charge_config_netket[len(charge_config_netket)//2:]  # down spins
# put nu 1s in the first half of the configuration (up spins) and shuffle
charge_config_netket_u[:nu] = 1  # assign nu up spins
rng.shuffle(charge_config_netket_u)  # shuffle the up spins to randomize their positions
# put nd 1s in the second half of the configuration (down spins) and shuffle
charge_config_netket_d[:nd] = 1  # assign nd down spins
rng.shuffle(charge_config_netket_d)  # shuffle the down spins to randomize their positions
# combine the up and down configurations back into a single netket configuration
charge_config_netket = np.concatenate((charge_config_netket_u, charge_config_netket_d))
charge_config = from_spinful_ind_to_charge(from_netket_config_to_quimb_config(charge_config_netket), symmetry='U1U1')
print(from_netket_config_to_quimb_config(charge_config_netket))
# charge_config_netket, sum(charge_config_netket[:len(charge_config_netket)//2]), sum(charge_config_netket[len(charge_config_netket)//2:]), charge_config

/global/u1/s/sijingdu/VMC/fermion/vmc_torch_experiment/vmc_torch/fermion_utils.py:256: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  new_oddpos = ftsdata.oddpos + (new_oddpos1,) if new_oddpos1 is not () else ftsdata.oddpos
/global/u1/s/sijingdu/VMC/fermion/vmc_torch_experiment/vmc_torch/fermion_utils.py:760: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  new_oddpos1 = FermionicOperator(new_oddpos, dual=True) if new_oddpos is not () else ()
/global/u1/s/sijingdu/VMC/fermion/vmc_torch_experiment/vmc_torch/fermion_utils.py:761: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  new_oddpos = ftsdata.oddpos + (new_oddpos1,) if new_oddpos1 is not () else ftsdata.oddpos


[2 2 1 2 0 0 3 2 1 0 3 1 3 1 0 3]


In [2]:
peps = qtn.TensorNetwork()
for site, info in sorted(site_info.items()):
    tid = site[0] * Ly + site[1]
    # bond index charge distribution
    block_indices = [
        sr.BlockIndex({(0, 0): d//4, (0, 1): d//4, (1, 0): d//4, (1, 1): d//4}, dual=dual)
        for d, dual in zip(info["shape"][:-1], info["duals"][:-1])
    ]
    # physical index
    p = info['shape'][-1]

    block_indices.append(
        sr.BlockIndex({(0, 0): p//4, (0, 1): p//4, (1, 0): p//4, (1, 1): p//4}, dual=info["duals"][-1])
    )

    data = sr.U1U1FermionicArray.random(
            block_indices,
            charge=charge_config[tid],
            seed=rng,
            oddpos=3*tid,
        )
    # *first_charges, last_charges = data.charges  # unpack the charges for logging
    peps |= qtn.Tensor(
            data=data,
            inds=info["inds"],
            tags=info["tags"],
        )

# required to view general TN as an actual PEPS
for i, j in site_info:
    peps[f"I{i},{j}"].add_tag([f"X{i}", f"Y{j}"])

peps.view_as_(
    fPEPS,
    site_ind_id="k{},{}",
    site_tag_id="I{},{}",
    x_tag_id="X{}",
    y_tag_id="Y{}",
    Lx=Lx,
    Ly=Ly,
)
peps = peps.copy() # set symmetry during initialization

In [3]:
from vmc_torch.fermion_utils import generate_random_fpeps

# SU in quimb
D = 4
seed = 0
symmetry = 'U1U1'
spinless = False
peps = generate_random_fpeps(Lx, Ly, D=D, seed=seed, symmetry=symmetry, Nf=nelec, spinless=spinless)[0]
edges = qtn.edges_2d_square(Lx, Ly, cyclic=False)
try:
    parse_edges_to_site_info = sr.utils.parse_edges_to_site_info
except AttributeError:
    parse_edges_to_site_info = sr.parse_edges_to_site_info
site_info = parse_edges_to_site_info(
    edges,
    D,
    phys_dim=4,
    site_ind_id="k{},{}",
    site_tag_id="I{},{}",
)

t = 1.0
U = 8.0
if nelec == int(Lx*Ly-2) or nelec == int(Lx*Ly-8):
    mu = 0.0 if symmetry in ['U1', 'U1U1'] else (U*nelec/(2*N)-2.42)#(U*nelec/(2*N)-2.3)
elif nelec == int(Lx*Ly):
    mu = 0.0 if symmetry in ['U1', 'U1U1'] else (U/2)
elif nelec == int(Lx*Ly-4):
    mu = 0.0 if symmetry in ['U1', 'U1U1'] else (U/2)-U*0.3
else:
    mu = 0.0

print(mu)

terms = {
    (sitea, siteb): sr.fermi_hubbard_local_array(
        t=t, U=U, mu=mu,
        symmetry=symmetry,
        coordinations=(
            site_info[sitea]['coordination'],
            site_info[siteb]['coordination'],
        ),
    )
    for (sitea, siteb) in peps.gen_bond_coos()
}
N_terms = {
    site: sr.fermi_number_operator_spinful_local_array(
        symmetry=symmetry
    )
    for site in peps.gen_site_coos()
}
occ_fn = lambda su: print(f'N per site:{su.get_state().compute_local_expectation(N_terms, normalized=True, max_bond=64,)/int(Lx*Ly)}') if su.n%50==0 else None

ham = qtn.LocalHam2D(Lx, Ly, terms)

su = qtn.SimpleUpdateGen(peps, ham, compute_energy_per_site=True,D=D, compute_energy_opts={"max_distance":1}, gate_opts={'cutoff':1e-12}, callback=occ_fn)

# cluster energies may not be accuracte yet
su.evolve(25, tau=0.3)
# su.evolve(50, tau=0.1)
# su.evolve(50, tau=0.03)
# # su.evolve(50, tau=0.01)
# # su.evolve(50, tau=0.003)

peps = su.get_state()
peps.equalize_norms_(value=1)

# save the state
params, skeleton = qtn.pack(peps)
    

0.0


n=25, tau=0.3, max|dS|=1: 100%|##########| 25/25 [00:09<00:00,  2.98it/s]   /global/homes/s/sijingdu/VMC/fermion/mpsds/mpsds/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
n=25, tau=0.3, max|dS|=1, energy~-0.28226: 100%|##########| 25/25 [00:11<00:00,  2.23it/s]


In [79]:
charge_config = np.zeros(Lx*Ly, dtype=int)

charge_config_netket = from_quimb_config_to_netket_config(charge_config)
charge_config_netket_u = charge_config_netket[:len(charge_config_netket)//2]  # up spins
charge_config_netket_d = charge_config_netket[len(charge_config_netket)//2:]  # down spins
# put nu 1s in the first half of the configuration (up spins) and shuffle
charge_config_netket_u[:nu] = 1  # assign nu up spins
rng.shuffle(charge_config_netket_u)  # shuffle the up spins to randomize their positions
# put nd 1s in the second half of the configuration (down spins) and shuffle
charge_config_netket_d[:nd] = 1  # assign nd down spins
rng.shuffle(charge_config_netket_d)  # shuffle the down spins to randomize their positions
# combine the up and down configurations back into a single netket configuration
charge_config_netket = np.concatenate((charge_config_netket_u, charge_config_netket_d))
charge_config = from_spinful_ind_to_charge(from_netket_config_to_quimb_config(charge_config_netket), symmetry='U1U1')
random_arr = from_netket_config_to_quimb_config(charge_config_netket)
print(random_arr)
print(peps.get_amp(random_arr, efficient=False).contract())
print(peps.get_amp(random_arr, efficient=True).contract())
# count how many 3s in random_arr

[1 1 0 1 1 2 2 3 3 2 0 2 3 0 2 1]
-1.2005917933861655e-08
-1.2005917933861655e-08


In [ ]:
from vmc_torch.fermion_utils import generate_random_fpeps
# # seed = np.random.randint(0, 2**32)  # Random seed for reproducibility
seed = 2
peps = generate_random_fpeps(Lx, Ly, D=D, seed=seed, symmetry='U1U1', Nf=int(Lx*Ly), spinless=False)[0]
# peps.get_amp(random_arr).contract()

StopIteration: 